In [1]:
import cv2
import os
import numpy as np
import glob
import math
import scipy
from scipy.spatial import distance
from scipy import signal
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn import metrics

In [22]:
##Lấy danh sách mẫu mống mắt
cwd = os.getcwd()
for dir_path, dir_names, file_names in os.walk(cwd):
    for f in file_names:
        list_file = open("list_file.txt","a+")
        temp=str(os.path.join(dir_path,f)+'\n')
        if '.bmp' in temp:
            list_file.write(temp)
list_file = open("list_file.txt","r")
images_train=[]
images_test=[]
ID_images_train=[]
ID_images_test=[]
if list_file.mode == 'r':
    data =list_file.readlines()
    for i in data[:200]:
        if i[-6:-5]=='1':
            ID_images_test.append(i[-10:-5])
            images_test.append(cv2.imread(i[:-1]))
        else:
            ID_images_train.append(i[-10:-5])
            images_train.append(cv2.imread(i[:-1]))

In [23]:
print(ID_images_test)

['001_1', '002_1', '003_1', '004_1', '005_1', '006_1', '007_1', '008_1', '009_1', '010_1', '011_1', '012_1', '013_1', '014_1', '015_1', '016_1', '017_1', '018_1', '019_1', '020_1', '021_1', '022_1', '023_1', '024_1', '025_1', '026_1', '027_1', '028_1', '029_1', '030_1', '031_1', '032_1', '033_1', '034_1', '035_1', '036_1', '037_1', '038_1', '039_1', '040_1']


In [24]:
print(ID_images_train)

['001_2', '001_3', '001_4', '001_5', '002_2', '002_3', '002_4', '002_5', '003_2', '003_3', '003_4', '003_5', '004_2', '004_3', '004_4', '004_5', '005_2', '005_3', '005_4', '005_5', '006_2', '006_3', '006_4', '006_5', '007_2', '007_3', '007_4', '007_5', '008_2', '008_3', '008_4', '008_5', '009_2', '009_3', '009_4', '009_5', '010_2', '010_3', '010_4', '010_5', '011_2', '011_3', '011_4', '011_5', '012_2', '012_3', '012_4', '012_5', '013_2', '013_3', '013_4', '013_5', '014_2', '014_3', '014_4', '014_5', '015_2', '015_3', '015_4', '015_5', '016_2', '016_3', '016_4', '016_5', '017_2', '017_3', '017_4', '017_5', '018_2', '018_3', '018_4', '018_5', '019_2', '019_3', '019_4', '019_5', '020_2', '020_3', '020_4', '020_5', '021_2', '021_3', '021_4', '021_5', '022_2', '022_3', '022_4', '022_5', '023_2', '023_3', '023_4', '023_5', '024_2', '024_3', '024_4', '024_5', '025_2', '025_3', '025_4', '025_5', '026_2', '026_3', '026_4', '026_5', '027_2', '027_3', '027_4', '027_5', '028_2', '028_3', '028_4', 

In [25]:
def ImageEnhancement(normalized):
    enhanced=[]
    for res in normalized:
        res = res.astype(np.uint8)
        im=cv2.equalizeHist(res)
        enhanced.append(im)
    return enhanced

In [26]:
#modulating function as defined in paper
def m(x ,y, f):
    val = np.cos(2*np.pi*f*math.sqrt(x **2 + y**2))
    return val
#spatial filter as defined in paper
def gabor(x, y, dx, dy, f):
    gb = (1/(2*math.pi*dx*dy))*np.exp(-0.5*(x**2 / dx**2 + y**2 / dy**2)) * m(x, y, f)
    return gb

#function to calculate spatial filter over 8x8 blocks
def spatial(f,dx,dy):
    sfilter=np.zeros((8,8))
    for i in range(8):
        for j in range(8):
            sfilter[i,j]=gabor((-4+j),(-4+i),dx,dy,f)
    return sfilter

def get_vec(convolvedtrain1,convolvedtrain2):
    feature_vec=[]
    for i in range(6):
            for j in range(64):
                #Run 8 by 8 filtered block iteratively over the entire image
                start_height = i*8
                end_height = start_height+8
                start_wid = j*8
                end_wid = start_wid+8
                grid1 = convolvedtrain1[start_height:end_height, start_wid:end_wid]
                grid2 = convolvedtrain2[start_height:end_height, start_wid:end_wid]

                # Channel 1
                absolute = np.absolute(grid1)
                # mean
                mean = np.mean(absolute)
                feature_vec.append(mean)
                #deviation
                std = np.mean(np.absolute(absolute-mean))
                feature_vec.append(std)

                # Channel 2
                absolute = np.absolute(grid2)
                # mean
                mean = np.mean(absolute)
                feature_vec.append(mean)
                #deviation
                std = np.mean(np.absolute(absolute-mean))
                feature_vec.append(std)

    return feature_vec

def FeatureExtraction(enhanced):
    con1=[]
    con2=[]
    #get spatial filters
    filter1=spatial(0.67,3,1.5)
    filter2=spatial(0.67,4,1.5) 
    
    feature_vector=[]
    
    for i in range(len(enhanced)):
        img=enhanced[i]
        #define a 48x512 region over which the filters are applied
        img_roi=img[:48,:]
        
        filtered1=scipy.signal.convolve2d(img_roi,filter1,mode='same')
        filtered2=scipy.signal.convolve2d(img_roi,filter2,mode='same')
        
        con1.append(filtered1)
        con2.append(filtered2)
        fv=get_vec(filtered1,filtered2)
        feature_vector.append(fv)
    return feature_vector #each feature vector has a dimension of 1536



In [27]:
images_train=[ImageEnhancement(i) for i in images_train]
train=[FeatureExtraction(i) for i in images_train]

In [28]:
images_test=[ImageEnhancement(i) for i in images_test]
test=[FeatureExtraction(i) for i in images_test]

In [29]:
feature_vector_test=test
feature_vector_train=train

In [32]:
def score(feature_vector_train,feature_vector_test):
    score=0
    for i in range(0,1536):
        if np.isnan(feature_vector_train[i])==True and np.isnan(feature_vector_test[i])==True:
            score+=0
        elif np.isnan(feature_vector_train[i])==True:
            score+=feature_vector_test[i]
        elif np.isnan(feature_vector_test[i])==True:
            score+=feature_vector_train[i]
        else:
            score+=abs(feature_vector_train[i]-feature_vector_test[i])
    return score

In [37]:
a=[[sum([score(feature_vector_train[i][j],feature_vector_test[k][j]) for j in range(0,48)]) for i in range(0,160)] for k in range(0,40)]

In [67]:
a

[[490.29045753797715,
  363.0019985824758,
  475.77834732204747,
  620.8078984333899,
  743.5272749382693,
  709.7164234251151,
  712.5757567155755,
  748.562617987609,
  842.0095256045652,
  832.4434945398456,
  720.1447226499416,
  812.4259934358033,
  897.6868111631824,
  932.8905072040395,
  1046.3220096872142,
  981.2241490718739,
  738.5538566927289,
  839.5785030834994,
  800.5306353754006,
  786.8336302554821,
  815.4314229117681,
  809.6154029006769,
  824.3844620333202,
  790.1265552116627,
  882.542179953714,
  924.367660216278,
  780.4382023023461,
  778.824035497171,
  682.4276336558701,
  651.6414753134301,
  685.0570741394072,
  669.7693788421882,
  649.5142877445752,
  845.9881484429808,
  724.8072270398008,
  725.6471738162028,
  997.4152860773587,
  948.037503777431,
  932.2146080535852,
  800.1176006897305,
  724.6363805830792,
  695.5338367415803,
  690.4283686286877,
  719.6953937541464,
  837.6297038738646,
  840.4298386564896,
  841.5489577756528,
  775.704031689

In [59]:
result=[ID_images_train[i.index(min(i))][0:3] for i in a]

In [65]:
print(result)

['001', '002', '002', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '026', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '002', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040']


In [61]:
ID_images_test=[i[0:3] for i in ID_images_test ]

In [62]:
print(ID_images_test)

['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040']


In [63]:
from scipy.spatial import distance

In [66]:
1-distance.hamming(result,ID_images_test)

0.925